<a href="https://colab.research.google.com/github/yeonjin99/TIL/blob/main/pdf_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. PDF-to-Text
PDF 파일에서 언어 모델이 이해할 수 있는 플레인 텍스트 추출

**PDF-to-Image**, **Image-to-Text** 로직이 포함

## 1.1 PDF-to-Image
pdf2image 사용

#### 패키지 설명
- poppler-utils: PDF 파일을 다루기 위한 여러 유틸리티를 포함하는 패키지. PDF 파일에서 텍스트를 추출하거나, PDF 파일을 다른 형식으로 변환할 수 있는 도구들이 포함. ```pdfinfo```, ```pdftotext```, ```pdftohtml``` 등의 명령어를 사용할 수 있음

- pdf2image: **PDF 파일을 이미지로 변환** 하는 Python 패키지. PDF 파일의 각 페이지를 이미지 형식(PNG, JPEG ...)으로 변환하는 데 사용됨

In [ ]:
!apt-get install poppler-utils

In [ ]:
!pip install pdf2image

In [ ]:
# pdf2image 라이브러리 임포트
from pdf2image import convert_from_path

# 로컬 내 PDF 파일 경로 변수로 지정
FILE_NAME = "PATH_OF_PDF_FILE.pdf"

# 'convert_from_path' 함수 통해 PDF 파일 읽어와 이미지 리스트로 변환
images = convert_from_path(FILE_NAME)

In [ ]:
len(images)

In [ ]:
images

In [ ]:
# 변환된 이미지 확인
images[1]

In [ ]:
# 다음 단계를 위해 이미지 파일 로컬에 저장
for i, image in enumerate(images):
    image.save(f"page_{str(i)}.jpg", "JPEG")

## 1.2 Image-to-Text
Image-to-Text는 앞서 저장한 이미지 파일에서 텍스트를 추출하는 단계입니다.
Google OCR을 활용

In [ ]:
!pip install opencv-contrib-python
!pip install --upgrade google-cloud-vision
!pip install --upgrade google-cloud-speech
!pip install --upgrade google-cloud-language
!pip install --upgrade google-cloud-texttospeech

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print("Texts:")

    for text in texts:
        print(f'\n"{text.description}"')

        vertices = [
            f"({vertex.x},{vertex.y})" for vertex in text.bounding_poly.vertices
        ]

        print("bounds: {}".format(",".join(vertices)))

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )


In [ ]:
import io
from tqdm import tqdm
from google.cloud import vision

client = vision.ImageAnnotatorClient()

In [ ]:
detect_text("page_1.jpg").text